Dou et al. (2020) model

Agora em Julia versão 1.5.3

This script simulates the model

In [2]:
using LinearAlgebra, Statistics

# using Distributions, Expectations, NLsolve, Roots, Random, Plots, Parameters


using DataFrames
using BenchmarkTools
using GLM


In [3]:
using NBInclude
@nbinclude("solve_model.ipynb")

#importou a função solve_tree

In [4]:
#Vh/D, L/D, Dj/D
data1 = [0.86, 0.19, 0.8];
n1 = 49

data2 = [2.65, 0.69, 0.74];
n2 = 17

#ρ, β, c0, λj
game_parameters = [0.884, 9.84, 0.044, 0.346];

In [5]:
# s_W1, j_W1, Pst_array1, Pjt_array1, Ds1, Dj1 = solve_tree(data1, game_parameters);

In [6]:
A1 = solve_tree(data1, game_parameters);

117.067155 seconds (403.22 M allocations: 8.976 GiB, 1.25% gc time)


In [7]:
typeof(A1)

Tuple{Array{Float64,4},Array{Float64,4},Array{Float64,4},Array{Float64,4},Float64,Float64}

In [8]:
# s_W2, j_W2, Pst_array2, Pjt_array2, Ds2, Dj2 = solve_tree(data2, game_parameters);

In [9]:
A2 = solve_tree(data2, game_parameters);

 96.879341 seconds (344.10 M allocations: 7.634 GiB, 1.24% gc time)


Simulation parameters

In [22]:
#θs0, θj0, μ
simulation_parameters = [28, 36, 4.566];

### Criando funções para simular o jogo

In [23]:


# θs0, θj0, μ = simulation_parameters;
# θs0 = Int64(θs0);
# θj0 = Int64(θj0);

# #parameters to some functions that are similar as the ones in solve_model

# c0 = game_parameters[3];
# c1 = Float64(0.0015);

# ρ = game_parameters[1];
# λj = game_parameters[4];


# Vmax1 = data1[1];
# L1 = data1[2];
# D1 = Ds1 + Dj1;


In [24]:
# Vmax2 = data2[1];
# L2 = data2[2];
# D2 = Ds2 + Dj2;

In [25]:
#cost function
function Ct(t, c0=c0, c1=c1)
    
    #didn't use D because is normalized to D == 1.0
    #cost at period t=0(index1) is 0
    if(t <= 1)
        return 0
    else
        return c0 + c1 * (t-1) #test to make index==1 be t==0
    end
end



#liquidation payoffs
function s_L(t, L, Ds, Dj, c0, c1)
    return min(L - Ct(t, c0, c1), Ds)
end


function j_L(t, L, Ds, Dj, c0, c1)
    return min(L - Ct(t, c0, c1) - s_L(t, L, Ds, Dj, c0, c1), Dj)
end


function proposal(Pkt_array, t, hkt, lmt)
    
    #pkt array será sempre do propositor, quem responder às propostas apenas olhará o seu valor de continuação
    
    return policy, payoff_prop, payment, lm_next = Pkt_array[t, hkt, lmt, [end,end-1, 1, 2]]
    
end
    

#maximum value of reorganization each period
function Vt(Vmax, ρ, t)

    if(t <=1)
        return Vmax
    else
        #(t-2) instead of (t-1) because we shifted the indexes in the game so as to include t==0 at index==1
        return ρ^(t-2) * Vmax
    end


end

# answer_liq
function answer_liq(m_L, t, hm_next, payoff_prop, L, Ds, Dj, Vmax, ρ, c0, c1)
    
    liq = (m_L(t, L, Ds, Dj, c0, c1), Vt(Vmax, ρ, t) * hm_next/100 - Ct(t, c0, c1) - payoff_prop)
    
    payoff_liq, answer = findmax(liq)
    
    return payoff_liq, answer
end

function answer_reorg(payment, Cont_val, t, hm_next, lm_next, lk_next)
    
    reorg_value = (payment, Cont_val[t+1, hm_next, lm_next, lk_next])

    payoff_reorg, answer = findmax(reorg_value)
    
    return payoff_reorg, answer
end

function update_beliefs(propositor, hk_next, hm_next, lk_next, lm_next)
    
    if(propositor=="s")
        hst = hk_next
        lst = lk_next
        
        hjt = hm_next
        ljt = lm_next
        
    else
        hst = hm_next
        lst = lm_next
        
        hjt = hk_next
        ljt = lk_next
    end
    
    
    return hst, lst, hjt, ljt
end

        

update_beliefs (generic function with 1 method)

In [26]:
#código para tirar um draw da beta

function draw_beta(hkt, β=game_parameters[2], grid=100)

    u = rand()

    if(hkt == grid)
        
        return grid

    else
        
        x = 1.0 - exp(1.0/β * (log(1.0 - u) + β * log(1.0 - hkt/grid)))
        x = round(x * 100, digits=0)
#         return Int64(x * 100) #to convert in an integer
        return Int64(x)
        
    end
end

        

# @code_warntype draw_beta(10)



draw_beta (generic function with 3 methods)

In [27]:
function choose_parameters(propositor, hst, lst, hjt, ljt, Pst_array, Pjt_array, s_W, j_W)
    
    if(propositor=="s")
        
        Pkt_array = Pst_array
        Cont_val = j_W
        prop_index = 1
        respondent_index = 2 #índice de j, para organizar o payoff
        m_L = j_L

        hkt = hst
        lkt = lst
        
        hmt = hjt
        lmt = ljt


        lk_next = hkt

        hk_next = draw_beta(hkt)
        hm_next = draw_beta(hmt)
        
    elseif(propositor=="j")
        Pkt_array = Pjt_array
        Cont_val = s_W
        prop_index = 2
        respondent_index = 1
        m_L = s_L

        hkt = hjt
        lkt = ljt
        
        hmt = hst
        lmt = lst


        lk_next = hkt


        hk_next = draw_beta(hkt)
        hm_next = draw_beta(hmt)

    else
        
        println("error: propositor not valid")
        
    end
    
    return Pkt_array, Cont_val, prop_index, respondent_index, m_L, hkt, lkt, hmt, lmt, lk_next, hk_next, hm_next
end




# #test
# Hs0 = 30
# Hj0 = 28


# hst = Hs0
# hjt = Hj0

# #assumindo que lower bounds nos períodos iniciais são as próprias habilidades iniciais
# lst = hst
# ljt = hjt




# choose_parameters("s", hst, lst, hjt, ljt);
    
    

In [28]:
function simulate_game(data, game_parameters, simulation_parameters, s_W=s_W, j_W=j_W, Pst_array=Pst_array, Pjt_array=Pjt_array, t=1, grid=100)
    
    
    Vmax = data[1]
    L = data[2]
    Dj = data[3]
    Ds = one(1.0) - Dj
    
    
    c0 = game_parameters[3];
    c1 = Float64(0.0015);

    ρ = game_parameters[1];
    λj = game_parameters[4];

    
    
#     hst = Hs0
#     hjt = Hj0
    
    hst = Int64(simulation_parameters[1])
    hjt = Int64(simulation_parameters[2])

    #assumindo que lower bounds nos períodos iniciais são as próprias habilidades iniciais
    lst = hst
    ljt = hjt
    
    result = zeros(5);
    
    #number of observed proposals
    observed_proposals = 0.0
    
    
    #recovering "T+1" from s_W 
    T = size(s_W,1)
    
    
    while(result[1]==0.0 && t < T)
        
        
        u = rand()

        if(u < λj)
            propositor = "j"

        else
            propositor = "s"

        end

        
        #setting the default variables according to the propositor
        Pkt_array, Cont_val, prop_index, respondent_index, m_L, hkt, lkt, hmt, lmt, lk_next, hk_next, hm_next = choose_parameters(propositor, hst, lst, hjt, ljt, Pst_array, Pjt_array, s_W, j_W)
        
        #proposal ####
        policy, payoff_prop, payment, lm_next = proposal(Pkt_array, t, hkt, lmt)

        #lm_next is the update of the adversary's lower bound
        if(lm_next>=grid+1 || policy!= 3.0) #para evitar update de lowerbound quando proponente não propõe reorg
            lm_next = lmt
        else
            lm_next = Int64(lm_next)
        end
        
        
        
        if(policy==3.0)
            
            observed_proposals += 1.0
    
            payoff_respondent, answer = answer_reorg(payment, Cont_val, t, hm_next, lm_next, lk_next)


            if(answer==1.0)

                result[prop_index] = payoff_prop
                result[respondent_index] = payoff_respondent
                result[3] = 3.0
                result[4] = t
                result[5] = observed_proposals

            else
                
                t+=1

                hst, lst, hjt, ljt = update_beliefs(propositor, hk_next, hm_next, lk_next, lm_next)
            end

        elseif(policy==2.0)
            
            t+=1
            
            hst, lst, hjt, ljt = update_beliefs(propositor, hk_next, hm_next, lk_next, lm_next)
            
            

        else      
            #(policy==1.0)
            
            observed_proposals += 1.0

            payoff_respondent, answer = answer_liq(m_L, t, hm_next, payoff_prop, L, Ds, Dj, Vmax, ρ, c0, c1)
            
            

            if(answer==1.0)

                result[prop_index] = payoff_prop
                result[respondent_index] = payoff_respondent
                result[3] = 1.0
                result[4] = t
                result[5] = observed_proposals

            else

                result[prop_index] = payoff_prop
                result[respondent_index] = payoff_respondent
                result[3] = 3.0
                result[4] = t
                result[5] = observed_proposals


            end
        end


        if(t==T)
            
            result[1] = s_W[T, 1, 1, 1]#todos os valores finais de S são iguais, então acessarei o índice 1
            result[2] = j_W[T, 1, 1, 1]
            result[3] = 1.0
            result[4] = t
            result[5] = max(observed_proposals, 1.0)
            
        end
        




    end
    
    
    return result
end

        
        
#precisa colocar uma condição IF aqui para caso o jogo vá até o último período?
    

simulate_game (generic function with 7 methods)

In [29]:
#unit testing
@btime simulate_game(data1, game_parameters, simulation_parameters, A1[1], A1[2], A1[3], A1[4])

  3.200 μs (19 allocations: 875 bytes)


5-element Array{Float64,1}:
 0.27398715628180514
 0.07191038953353354
 3.0
 3.0
 1.0

In [30]:
function loop_simulations(S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)
    
    
    
    
    
    #S is the number of simulations
    #N is the number of observations
    #data1 is the center of the first cluster
    #A1 is the collection of arrays from the first cluster (s_W, j_W, Pst, Pjt)
    #n1 is the number of observations in the first cluster
    
    
    
    
    #no futuro os argumentos podem ser arrays de arrays, assim ele faz o loop para cada cluster
    Results = zeros(S, N, 7)

    #últimas entradas de Results são os valores das dívidas
    
    
    for s in 1:S
        for n in 1:N
            if(n <= n1)
                Results[s,n,1:5] .= simulate_game(data1, game_parameters, simulation_parameters, A1[1], A1[2], A1[3], A1[4])
                Results[:,:, end-1] .= A1[5]
                Results[:,:, end] .= A1[6]
            else
                Results[s,n,1:5] .= simulate_game(data2, game_parameters, simulation_parameters, A2[1], A2[2], A2[3], A2[4])
                Results[:,:, end-1] .= A2[5]
                Results[:,:, end] .= A2[6]
            end
            
        end
    end

    return Results
    
end

    
# loop_simulations(40, 75, θs0, θj0, μ, λj);

In [31]:
#unit testing
S = 40
N = 66
Results = loop_simulations(S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)

40×66×7 Array{Float64,3}:
[:, :, 1] =
 0.163404  0.206137  0.213061  0.234006  …  0.436504  0.531209  0.416333
 0.196917  0.208813  0.261983  0.166285     0.481694  0.592079  0.445419
 0.215867  0.270502  0.219506  0.200496     0.557979  0.44436   0.673354
 0.228388  0.180361  0.25361   0.231083     0.293243  0.369152  0.466484
 0.213053  0.28675   0.224502  0.21397      0.454959  0.605309  0.350881
 0.224502  0.187897  0.224502  0.257269  …  0.451703  0.425411  0.473383
 0.227222  0.160413  0.179854  0.226541     0.434763  0.468971  0.263296
 0.227891  0.238149  0.315083  0.224502     0.648033  0.504816  0.72686
 0.215459  0.224502  0.254649  0.189024     0.339942  0.390409  0.372233
 0.224502  0.224502  0.145097  0.280249     0.491128  0.445143  0.384923
 0.215646  0.329913  0.210364  0.170804  …  0.590513  0.397007  0.502138
 0.224502  0.224502  0.194232  0.224502     0.327486  0.453778  0.453813
 0.224502  0.224502  0.224502  0.193574     0.587438  0.476162  0.504816
 ⋮            

In [32]:
function simulate_moments(S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)
    
    
    Results = loop_simulations(S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)
    
    Moments = zeros(S, 8)
    
    μ = simulation_parameters[3]
    for s in 1:S

            DF = DataFrame()
            DF.payoff_s = Results[s,:,1];
            DF.payoff_j = Results[s,:,2];
            DF.out = Results[s,:,3];
            DF.t = Results[s,:,4];
            DF.observed_proposals = Results[s,:,5];
            DF.Dj = Results[s,:,6];
            DF.Ds = Results[s,:,7];


            DF.outcome = ifelse.(DF.out .==3.0, "R", "L");


            #desfazendo o deslocamento no índice de t
            DF.t = DF.t .- 1.0;

            # DF.incourt = ifelse.(DF.t .> 0.0, "incourt", "precourt");

            INCOURT = filter(DF -> DF.t .> 0.0, DF);
            PRECOURT = filter(DF -> DF.t .<= 0.0, DF);

            PRECOURT_REORG = filter(PRECOURT -> PRECOURT.outcome .== "R", PRECOURT)
            INCOURT_REORG = filter(INCOURT -> INCOURT.outcome .== "R", INCOURT)

            #cálculo dos momentos
            #1. avg log number of months between observed proposals incourt

            mm1 = INCOURT
            mm1 = log.(mm1.t .* μ ./ mm1.observed_proposals)
            mm1 = mean(mm1)

            #2. fraction reorganized given that the case went into court

            mm2 = INCOURT
            mm2 = size(filter(mm2 -> mm2.outcome .== "R", mm2),1)/ size(mm2,1)

            #3. ln duration of court cases in months

            mm3 = INCOURT
            mm3 = filter(mm3 -> mm3.t .> 0.0, mm3) #removendo os casos 0 para não poluir a média
            mm3.t = mm3.t .* μ
            mm3 = mean(log.(mm3.t)) #log here uses exp as base, so it's the same as ln

            #4. fraction of cases incourt
            mm4 = size(INCOURT,1) / size(DF, 1)

            #5. avg recovery rate for senior given precourt REORGANIZATION

            #aqui nós temos Ds, então é mais fácil. Preciso automatizar isso no código quando for tudo escalado

            mm5 = PRECOURT_REORG
            mm5.R_s = mm5.payoff_s ./ mm5.Ds
            mm5 = mean(mm5.R_s)



            #6. avg recovery rate for junior given precourt REORGANIZATION

            #aqui nós temos Ds, então é mais fácil. Preciso automatizar isso no código quando for tudo escalado

            mm6 = PRECOURT_REORG
            mm6.R_j = mm6.payoff_j ./ mm6.Dj
            mm6 = mean(mm6.R_j)


            #7. junior avg fraction gain given incourt REORGANIZATION

            mm7 = INCOURT_REORG
            mm7 = mean(mm7.payoff_j ./ (mm7.payoff_j .+ mm7.payoff_s )) 

            #8. total recovery rate given incourt REORGANIZATION

            mm8 = INCOURT_REORG
            mm8 = mean( mm8.payoff_s .+ mm8.payoff_j)
        
        
            Moments[s,:] .= [mm1, mm2, mm3, mm4, mm5, mm6, mm7, mm8]

            
            #if there are no INCOURT or no PRECOURT cases, it will return NaN
            #thus, we replace it by zero
            Moments[s,:] .= replace!(Moments[s,:], NaN=> zero(0.0))

        end
    return vec(mean(Moments, dims=1))
end



  

simulate_moments (generic function with 1 method)

In [37]:
#test
S = 40
N = 66

simulated_moments = simulate_moments(S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)

8-element Array{Float64,1}:
 1.5433390724280858
 0.9995614035087719
 2.496624635286876
 0.8893939393939393
 0.3033808619430241
 0.47027914990989783
 0.3755280612161006
 0.5444020141904221

Calculando a derivada dos momentos em relação aos parâmetros

In [40]:
#data_moments
data_moments = [0.728, 0.843, 2.76, 0.969, 0.6235, 0.5569, 0.773, 0.497];

In [39]:
function simulate_new_moments(i, S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)
    
    
    println("parameter ",i, game_parameters, simulation_parameters)
        
    θs0, θj0, μ = simulation_parameters;
    θs0 = Int64(round(θs0, digits=0));
    θj0 = Int64(round(θj0, digits=0));

    λj = game_parameters[4];
    new_simulated_moments = simulate_moments(S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1) 
    
    return new_simulated_moments
end

simulate_new_moments (generic function with 1 method)

In [41]:
# calculando a derivada em relação a ρ

function derivative_moments(S, N, game_parameters, simulation_parameters, simulated_moments, data1, data2, n1)
    
    
    P = size(game_parameters,1)+size(simulation_parameters,1)
    M = size(simulated_moments,1)
    D = zeros(M, P)
    
    estimated_game_parameters = game_parameters
    estimated_simulation_parameters = simulation_parameters
    
    #delta for each parameter
    #approximately 1% or the lowest decimal point
    #[game_parameters, simulation_parameters]
    delta = [0.01, 0.22, 0.002, 0.01, 1.0, 1.0, 0.03]
    
    for i in 1:P
        if(i<=size(game_parameters,1))
            game_parameters = copy(estimated_game_parameters)
            
            game_parameters[i] = game_parameters[i] + delta[i]
            
            
        else
            simulation_parameters = copy(estimated_simulation_parameters)
            simulation_parameters[i-size(game_parameters,1)] = simulation_parameters[i-size(game_parameters,1)] + delta[i]
            
            
        end
        
        
        
        A1 = solve_tree(data1, game_parameters)
        A2 = solve_tree(data2, game_parameters)
        
        new_simulated_moments = simulate_new_moments(i, S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)
        
        D[:, i] = new_simulated_moments - simulated_moments
        
    end
    
    
    
    return D
    
end


derivative_moments (generic function with 1 method)

In [42]:
Derivative = derivative_moments(S, N, game_parameters, simulation_parameters, simulated_moments, data1, data2, n1)

130.865096 seconds (430.89 M allocations: 9.554 GiB, 1.29% gc time)
105.493785 seconds (401.96 M allocations: 8.914 GiB, 1.59% gc time)
parameter 1[0.894, 9.84, 0.044, 0.346][28.0, 36.0, 4.566]
111.238220 seconds (401.96 M allocations: 8.914 GiB, 1.20% gc time)
 91.731382 seconds (344.10 M allocations: 7.634 GiB, 1.27% gc time)
parameter 2[0.884, 10.06, 0.044, 0.346][28.0, 36.0, 4.566]
111.467502 seconds (401.96 M allocations: 8.914 GiB, 1.34% gc time)
 90.438843 seconds (344.10 M allocations: 7.634 GiB, 1.31% gc time)
parameter 3[0.884, 9.84, 0.046, 0.346][28.0, 36.0, 4.566]
113.781866 seconds (401.96 M allocations: 8.914 GiB, 1.16% gc time)
 90.520473 seconds (344.10 M allocations: 7.634 GiB, 1.23% gc time)
parameter 4[0.884, 9.84, 0.044, 0.356][28.0, 36.0, 4.566]
110.901104 seconds (401.96 M allocations: 8.914 GiB, 1.21% gc time)
104.744490 seconds (344.10 M allocations: 7.634 GiB, 1.48% gc time)
parameter 5[0.884, 9.84, 0.044, 0.356][29.0, 36.0, 4.566]
112.428968 seconds (401.96 M 

8×7 Array{Float64,2}:
  0.0905422    -0.00516918   0.00393462  …  -0.00816405  -0.0134551
 -0.000840871  -0.0012646   -0.00130952     -0.00133488  -0.000895427
  0.148458      0.00671346   0.024089        0.00370195  -0.00102116
  0.0094697    -0.00757576  -0.00984848     -0.0246212   -0.0155303
  0.000410421  -0.00246253  -0.00287295     -0.00127466  -0.00123126
 -0.00125287    0.00139177   0.00456686  …   0.0269693    0.00429738
 -0.00348118   -0.00199581  -0.00156134      0.00428559   0.000308983
  0.00121825    0.00169847  -0.00208221      0.0103047    0.00364449

In [43]:
Derivative[2,:]

7-element Array{Float64,1}:
 -0.0008408705483323731
 -0.0012645993618917162
 -0.0013095238095237605
 -0.0030372853706545078
 -4.2172739541213033e-5
 -0.001334875937981983
 -0.0008954269267668069